<a href="https://colab.research.google.com/github/shreyaghotankar/CMPE-255-Data-Mining/blob/master/CMPE255_ApproximateNearestNeighbor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment - Approximate nearest neighbor search

a) LSH

b) exhaustive search

c) product quantization

d) trees

e) hnsw

In [ ]:
import pickle
import pandas as pd
import numpy as np
import os
import json
import math
import statistics
import re

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

# Locality Sensitive Hashing (LSH)

## Using FAISS

**Dataset**: [Stackexchange Network - From StackOverflow](https://making.lyst.com/lightfm/docs/datasets.html)


Approximately 1.3M users, 11M questions, and 18M answers.

Info:
* train (sp.coo_matrix of shape [n_users, n_items]) – Contains training set interactions.
* test (sp.coo_matrix of shape [n_users, n_items]) – Contains testing set interactions.
* item_features (sp.csr_matrix of shape [n_items, n_item_features]) – Contains item features.
* item_feature_labels (np.array of strings of shape [n_item_features,]) – Labels of item features.

In [ ]:
!pip install lightfm

In [ ]:
from lightfm import LightFM
from lightfm.datasets import fetch_stackexchange

In [ ]:
stackexchange = fetch_stackexchange(dataset='stackoverflow') # stackexchange data where users answers questions like on stackoverflow.

Creating LightFM model and fit to the dataset to generate embeddings that will be used to create vectors.

In [ ]:
model = LightFM(loss='warp')
%time model.fit(stackexchange['train'], epochs=30, num_threads=2)

CPU times: user 28min 39s, sys: 1.57 s, total: 28min 41s
Wall time: 14min 52s


In [ ]:
item_vectors = stackexchange['item_features'] * model.item_embeddings

saving the vectors created in above step in pickle file

In [ ]:
with open('stackexchange.pickle', 'wb') as f:
    pickle.dump({"name": stackexchange['item_feature_labels'], "vector": item_vectors}, f)

In [ ]:
!pip install faiss-gpu # installing faiss gpu

     |████████████████████████████████| 89.7 MB 20 kB/s 


In [ ]:
import faiss

In [ ]:
# loading the pickled dataset with vectors
def load_data():
    with open('/content/drive/MyDrive/CMPE255 - Data Mining/Assignments/Datasets/stackexchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['question_id:0', 'question_id:1', 'question_id:2', ...,
        'question_id:11280893', 'question_id:11280894',
        'question_id:11280895'], dtype='<U20'),
 'vector': array([[ 0.01564435,  0.2614761 , -0.2828568 , ...,  0.6273946 ,
          0.6865856 , -0.27049625],
        [ 0.22873431, -0.00347478, -0.46082127, ...,  0.3264265 ,
          0.52347016, -0.12953286],
        [-0.3405572 ,  0.29287434, -0.24874553, ...,  0.8486006 ,
          0.25700596, -0.44324523],
        ...,
        [ 0.06665161, -0.09037501, -0.12953791, ..., -0.004171  ,
          0.22689764,  0.5356065 ],
        [-0.27515164,  0.2124995 ,  0.21542324, ..., -0.14622097,
          0.1833778 ,  0.12455014],
        [ 0.1305509 , -0.16130476,  0.13273829, ..., -0.02589074,
         -0.46399045,  0.12061999]], dtype=float32)}

Creating an index function that is used to add dimensions to the index.

In [ ]:
class LSH_Index():
     def __init__(self, vectors, labels):
         self.dimension = vectors.shape[1]
         self.vectors = vectors.astype('float32')
         self.labels = labels


     def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
         quantizer = faiss.IndexFlatL2(self.dimension)
         self.index = faiss.IndexIVFPQ(quantizer, self.dimension, number_of_partition, search_in_x_partitions, subvector_size)
         self.index.train(self.vectors)
         self.index.add(self.vectors)
        
     def query(self, vectors, k=10):
         distances, indices = self.index.search(vectors, k) 
         return [self.labels[i] for i in indices[0]]

In [ ]:
index = LSH_Index(data["vector"], data["name"])
index.build()

In [ ]:
stackexch_vector, stackexch_name = data['vector'][80:81], data['name'][80]
simlar_stackexch_names = '\n* '.join(index.query(stackexch_vector))
print(f"Similar stuff to {stackexch_name} are:\n* {simlar_stackexch_names}")

Similar stuff to question_id:80 are:
* question_id:4746
* question_id:21897
* question_id:60707
* question_id:159891
* question_id:80
* question_id:64190
* question_id:190187
* question_id:202659
* question_id:187688
* question_id:57529


The results are the most similar items to question id 80 in the dataset

## Using Datasketch and pinecone

**Dataset**: [NeurIPS conference papers](https://www.kaggle.com/rowhitswami/nips-papers-1987-2019-updated)

In [ ]:
!pip install nltk
!pip install gensim
!pip install transformers
!pip install -qU datasketch gensim mmh3 pinecone-client ipywidgets
!pip install -qU sentence-transformers --no-cache-dir

     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 41.9 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 47.7 MB/s 
     |████████████████████████████████| 895 kB 69.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 67 kB 2.5 MB/s 
     |████████████████████████████████| 24.1 MB 2.4 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 156 kB 73.2 MB/s 
     |████████████████████████████████| 57 kB 3.9 MB/s 
     |████████████████████████████████| 241 kB 58.8 MB/s 
     |████████████████████████████████| 140 kB 58.9 MB/s 
     |████████████████████████████████| 78 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 9.0 MB/s 


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')

import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df_papers = pd.read_csv('/content/drive/MyDrive/CMPE255 - Data Mining/Assignments/Datasets/papers.csv')
df_papers.head()

,source_id,year,title,abstract,full_text
0,27,1987,Bit-Serial Neural Networks,NaN,573 \n\nBIT - SERIAL NEURAL NETWORKS \n\nAlan...
1,63,1987,Connectivity Versus Entropy,NaN,1 \n\nCONNECTIVITY VERSUS ENTROPY \n\nYaser S...
2,60,1987,The Hopfield Model with Multi-Level Neurons,NaN,278 \n\nTHE HOPFIELD MODEL WITH MUL TI-LEVEL N...
3,59,1987,How Neural Nets Work,NaN,442 \n\nAlan Lapedes \nRobert Farber \n\nThe...
4,69,1987,Spatial Organization of Neural Networks: A Pro...,NaN,740 \n\nSPATIAL ORGANIZATION OF NEURAL NEn...


In [ ]:
df_papers.shape

(9680, 5)

In [ ]:
df_papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9680 entries, 0 to 9679
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   source_id  9680 non-null   int64 
 1   year       9680 non-null   int64 
 2   title      9680 non-null   object
 3   abstract   6361 non-null   object
 4   full_text  9677 non-null   object
dtypes: int64(2), object(3)
memory usage: 378.2+ KB


In [ ]:
df_papers.isna().sum() # checking for missing values

source_id       0
year            0
title           0
abstract     3319
full_text       3
dtype: int64

In [ ]:
df_papers.dropna(inplace=True) # dropping missing values
df_papers.shape

(6360, 5)

In [ ]:
df_papers.head()

,source_id,year,title,abstract,full_text
1321,5220,1997,Learning Generative Models with the Up Propaga...,Up-propagation is an algorithm for inverting ...,Learning Generative Models with the\n\nUp(cid:...
1334,5221,1997,A Neural Network Based Head Tracking System,We have constructed an inexpensive video based...,A Neural Network Based\n\nHead Tracking System...
1897,1861,2000,Algorithms for Non-negative Matrix Factorization,Non-negative matrix factorization (NMF) has pr...,Algorithms for Non-negative Matrix \n\nFactori...
1995,1975,2001,Characterizing Neural Gain Control using Spike...,Spike-triggered averaging techniques are effec...,Characterizing neural gain control using\n\nsp...
3126,195,2007,Compressed Regression,Recent research has studied the role of sparsi...,Compressed Regression\n\nShuheng Zhou∗ John La...


In [ ]:
stop_words = set(stopwords.words('english'))
porter_stemmer = PorterStemmer()
def preprocessText(text):
    text = text.lower()
    text = re.sub("[^a-z A-Z]", ' ', text)
    text = [porter_stemmer.stem(word) for word in text.split(' ') if not word in stop_words and word != '']
    return text


In [ ]:
from datasketch import MinHash, MinHashLSHForest

In [ ]:
def create_forest(df, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in df['abstract']:
        tokens = preprocessText(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [ ]:
permutations=120

forest = create_forest(df_papers, permutations)

It took 34.87200427055359 seconds to build forest.


In [ ]:
def predict(text, df, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocessText(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = df.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [ ]:
num_recommendations = 3
title = 'Efficient Rematerialization for Deep Networks'
result = predict(title, df_papers, permutations, num_recommendations, forest)
result

It took 0.006567955017089844 seconds to query forest.


5367                            Multi-Class Deep Boosting
6889    Positive-Unlabeled Learning with Non-Negative ...
7527      Collaborative Learning for Deep Neural Networks
Name: title, dtype: object

Using 120 permutations and requesting for 3 recommendations response, the above result was generated using the LSH forest.

# Exhaustive Search

In [ ]:
class ES_Index():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = ES_Index(data["vector"], data["name"])

In [ ]:
index.build()

In [ ]:
stackexch_vector, stackexch_name = data['vector'][80:81], data['name'][80]
simlar_stackexch_names = '\n* '.join(index.query(stackexch_vector))
print(f"Similar questions to {stackexch_name} are:\n* {simlar_stackexch_names}")

Similar questions to question_id:80 are:
* question_id:80
* question_id:914340
* question_id:936984
* question_id:458799
* question_id:1290021
* question_id:381947
* question_id:517705
* question_id:960165
* question_id:431473
* question_id:1256271


We can compare the LSH and Exhaustive Search response for the same dataset and for the same question id and we can easily difference in the output generated by both the approaches.

# Product Quantization

In [ ]:
import faiss

Implementing using the Inverted File

In [ ]:
class prodQuantIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, self.dimension, number_of_partition, search_in_x_partitions, subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = prodQuantIndex(data["vector"], data["name"])
index.build()

In [ ]:
question_index = 80
stackexch_vector = data['vector'][question_index:question_index+1]
print(f"simillar questions to {data['name'][question_index]} are:")
index.query(stackexch_vector)

simillar questions to question_id:80 are:


['question_id:4746',
 'question_id:21897',
 'question_id:60707',
 'question_id:159891',
 'question_id:80',
 'question_id:64190',
 'question_id:190187',
 'question_id:202659',
 'question_id:187688',
 'question_id:57529']

This response is close to LSH recommendation than Exhaustive search

# Tree

[Using California Fire dataset](https://www.kaggle.com/ananthu017/california-wildfire-incidents-20132020)

Implementing the tree algorithms for nearest neighbor using sklearn.neighbors library

In [ ]:
from sklearn.neighbors import BallTree, KDTree
from sklearn.model_selection import train_test_split

In [ ]:
df_fire = pd.read_csv('/content/drive/MyDrive/CMPE255 - Data Mining/Assignments/Datasets/California_Fire_Incidents.csv')
df_fire

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,Featured,Final,FuelType,Helicopters,Injuries,Latitude,Location,Longitude,MajorIncident,Name,PercentContained,PersonnelInvolved,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2013-09-06T18:30:00Z,NaN,False,True,NaN,NaN,NaN,37.857000,3 miles east of Groveland along Hwy 120,-120.086000,False,Rim Fire,100.0,NaN,True,The Rim Fire was east of Groveland along Highw...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,19,NaN,NaN,NaN,2013-06-08T18:30:00Z,NaN,False,True,NaN,NaN,NaN,34.585595,Angeles National Forest,-118.423176,False,Powerhouse Fire,100.0,NaN,True,The Powerhouse Fire burned in May and June 201...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,33,NaN,NaN,NaN,2013-07-30T18:00:00Z,NaN,False,True,NaN,NaN,NaN,33.709500,Hwy 243 & Hwy 74 near Mountain Center,-116.728850,False,Mountain Fire,100.0,NaN,True,The Mountain Fire burned in July 2013 off High...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,31,NaN,NaN,NaN,2013-08-30T08:00:00Z,NaN,False,True,NaN,NaN,NaN,39.120000,"Deadwood Ridge, northeast of Foresthill",-120.650000,False,American Fire,100.0,NaN,True,The American Fire burned in August 2013 off De...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,56,47.0,8.0,117.0,2013-05-11T06:30:00Z,NaN,False,True,NaN,11.0,10.0,0.000000,Southbound Highway 101 at Camarillo Springs Ro...,0.000000,True,Springs Fire,100.0,2167.0,True,"The Springs Fire burned in May 2013, Southboun...","Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,9.0,False,CAL FIRE / Riverside County Fire,NaN,2019,True,/incidents/2019/10/10/eagle-fire/,NaN,NaN,Riverside,33,NaN,NaN,NaN,2019-10-10T18:11:00Z,NaN,False,True,NaN,NaN,NaN,33.827979,"Eagle Canyon Rd. and Cajalco Rd., southwest of...",-117.499619,False,Eagle Fire,100.0,NaN,True,"The Eagle Fire started on October 10, 2019 nea...",NaN,2019-10-10T12:08:00Z,Finalized,NaN,NaN,NaN,NaN,6e93b252-99a3-4214-9921-238373b17535,2019-10-10T18:11:59.733Z,NaN
1632,2.0,False,CAL FIRE Nevada-Yuba-Placer Unit,NaN,2019,True,/incidents/2019/6/28/long-fire/,NaN,NaN,Nevada,29,NaN,NaN,NaN,2019-06-28T17:33:00Z,NaN,False,True,NaN,NaN,NaN,39.409722,"Off of Long Point Road and Old Mill Road, Sou...",-121.000556,False,Long Fire,100.0,NaN,True,"Long Fire started on June 28, 2019 off of Long...",NaN,2019-06-28T15:03:04Z,Finalized,NaN,NaN,NaN,NaN,b38c0563-b32

In [ ]:
df_fire.shape

(1636, 40)

In [ ]:
df_fire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AcresBurned           1633 non-null   float64
 1   Active                1636 non-null   bool   
 2   AdminUnit             1636 non-null   object 
 3   AirTankers            28 non-null     float64
 4   ArchiveYear           1636 non-null   int64  
 5   CalFireIncident       1636 non-null   bool   
 6   CanonicalUrl          1636 non-null   object 
 7   ConditionStatement    284 non-null    object 
 8   ControlStatement      111 non-null    object 
 9   Counties              1636 non-null   object 
 10  CountyIds             1636 non-null   object 
 11  CrewsInvolved         171 non-null    float64
 12  Dozers                123 non-null    float64
 13  Engines               191 non-null    float64
 14  Extinguished          1577 non-null   object 
 15  Fatalities           

In [ ]:
df_fire.isna().sum() # checking for missing values

AcresBurned                3
Active                     0
AdminUnit                  0
AirTankers              1608
ArchiveYear                0
CalFireIncident            0
CanonicalUrl               0
ConditionStatement      1352
ControlStatement        1525
Counties                   0
CountyIds                  0
CrewsInvolved           1465
Dozers                  1513
Engines                 1445
Extinguished              59
Fatalities              1615
Featured                   0
Final                      0
FuelType                1624
Helicopters             1552
Injuries                1516
Latitude                   0
Location                   0
Longitude                  0
MajorIncident              0
Name                       0
PercentContained           3
PersonnelInvolved       1432
Public                     0
SearchDescription         17
SearchKeywords           203
Started                    0
Status                     0
StructuresDamaged       1569
StructuresDest

for this assignment section working only with AdminUnit, Latitude and Longitude columns which do not have any missing values

In [ ]:
col_name = ['AdminUnit', 'Latitude', 'Longitude']
df_Calfire = df_fire[col_name].copy()

In [ ]:
df_Calfire

,AdminUnit,Latitude,Longitude
0,Stanislaus National Forest/Yosemite National Park,37.857000,-120.086000
1,USFS Angeles National Forest/Los Angeles Count...,34.585595,-118.423176
2,CAL FIRE Riverside Unit / San Bernardino Natio...,33.709500,-116.728850
3,Tahoe National Forest,39.120000,-120.650000
4,Ventura County Fire/CAL FIRE,0.000000,0.000000
...,...,...,...
1631,CAL FIRE / Riverside County Fire,33.827979,-117.499619
1632,CAL FIRE Nevada-Yuba-Placer Unit,39.409722,-121.000556
1633,Yolo County Fire Protection District,38.734634,-121.729691
1634,Camp Pendleton Marine Corps Base,33.351145,-117.403719


In [ ]:
# splitting the dataset into 2 - one used to build the tree and query will be run on the second.

X, y = train_test_split(df_Calfire, test_size=0.25,shuffle=False)
y = y.reset_index(drop=True)

## KD tree

In [ ]:
# implementing K-D tree
kd = KDTree(X[['Latitude', 'Longitude']].values, metric='euclidean')

dist, idx = kd.query(y[['Latitude', 'Longitude']].values, k=2) # k= 2 - neigbors to be returned

In [ ]:
print(KDTree.valid_metrics)

['euclidean', 'l2', 'minkowski', 'p', 'manhattan', 'cityblock', 'l1', 'chebyshev', 'infinity']


Merging the results in the y dataset to see the nearest neighbors

In [ ]:
dist_df = pd.DataFrame(dist, columns=['dist1', 'dist2'])

In [ ]:
df_result = y.merge(dist_df, how='outer', left_index=True, right_index=True)
df_result

,AdminUnit,Latitude,Longitude,dist1,dist2
0,CAL FIRE Mendocino Unit,39.279833,-123.258319,0.036804,0.097086
1,CAL FIRE Amador - El Dorado Unit,38.821280,-121.039180,0.044439,0.079454
2,CAL FIRE San Benito-Monterey,36.648320,-121.032530,0.013877,0.150191
3,CAL FIRE Santa Clara Unit/San Jose City Fire &...,37.176944,-121.686944,0.030872,0.059638
4,CAL FIRE Shasta-Trinity Unit,40.741186,-122.649626,0.037686,0.090728
...,...,...,...,...,...
404,CAL FIRE / Riverside County Fire,33.827979,-117.499619,0.030819,0.041702
405,CAL FIRE Nevada-Yuba-Placer Unit,39.409722,-121.000556,0.047608,0.130996
406,Yolo County Fire Protection District,38.734634,-121.729691,0.292456,0.297924
407,Camp Pendleton Marine Corps Base,33.351145,-117.403719,0.005682,0.051282


In [ ]:
adminUnit_df = pd.DataFrame(idx, columns=['unit1', 'unit2'])
df_result = df_result.merge(adminUnit_df, how='outer', left_index=True, right_index=True)
df_result

,AdminUnit,Latitude,Longitude,dist1,dist2,unit1,unit2
0,CAL FIRE Mendocino Unit,39.279833,-123.258319,0.036804,0.097086,700,631
1,CAL FIRE Amador - El Dorado Unit,38.821280,-121.039180,0.044439,0.079454,557,836
2,CAL FIRE San Benito-Monterey,36.648320,-121.032530,0.013877,0.150191,1147,539
3,CAL FIRE Santa Clara Unit/San Jose City Fire &...,37.176944,-121.686944,0.030872,0.059638,813,592
4,CAL FIRE Shasta-Trinity Unit,40.741186,-122.649626,0.037686,0.090728,985,1058
...,...,...,...,...,...,...,...
404,CAL FIRE / Riverside County Fire,33.827979,-117.499619,0.030819,0.041702,753,1018
405,CAL FIRE Nevada-Yuba-Placer Unit,39.409722,-121.000556,0.047608,0.130996,1027,115
406,Yolo County Fire Protection District,38.734634,-121.729691,0.292456,0.297924,1119,1118
407,Camp Pendleton Marine Corps Base,33.351145,-117.403719,0.005682,0.051282,697,692


In [ ]:
# get the name of the admin unit from X dataset based on index
def get_adminunit(ind):
  val = X.iloc[ind,:]
  au = val[0]
  return au


In [ ]:
df_result['adminUnit1'] = df_result['unit1'].apply(get_adminunit)

In [ ]:
df_result['adminUnit2'] = df_result['unit2'].apply(get_adminunit)

In [ ]:
df_result.drop(columns=['unit1','unit2'],inplace=True)

In [ ]:
df_result.head(15)

,AdminUnit,Latitude,Longitude,dist1,dist2,adminUnit1,adminUnit2
0,CAL FIRE Mendocino Unit,39.279833,-123.258319,0.036804,0.097086,CAL FIRE Mendocino Unit,CAL FIRE Mendocinio Unit
1,CAL FIRE Amador - El Dorado Unit,38.821280,-121.039180,0.044439,0.079454,CAL FIRE Amador-El Dorado Unit,CAL FIRE Amador-El Dorado Unit
2,CAL FIRE San Benito-Monterey,36.648320,-121.032530,0.013877,0.150191,CAL FIRE San Benito-Monterey,CAL FIRE San Benito Monterey Unit
3,CAL FIRE Santa Clara Unit/San Jose City Fire &...,37.176944,-121.686944,0.030872,0.059638,CAL FIRE Santa Clara Unit,CAL FIRE Santa Clara Unit
4,CAL FIRE Shasta-Trinity Unit,40.741186,-122.649626,0.037686,0.090728,CAL FIRE Siskiyou Unit,"Unified Command: CAL FIRE Shasta-Trinity Unit,..."
5,CAL FIRE / Riverside County Fire,33.563490,-116.732032,0.082713,0.093004,San Bernardino National Forest,CAL FIRE/Riverside County Fire/San Bernardino ...
6,Klamath National Forest,41.833590,-122.992720,0.126484,0.167671,USFS Klamath National Forest,Rogue-River-Siskiyou National Forest
7,CAL FIRE Santa Clara Unit,37.327440,-121.109030,0.113746,0.114004,CAL FIRE Madera-Mariposa-Merced Unit,CAL FIRE Madera-Mariposa-Merced Unit
8,Kern County Fire Department,34.860220,-118.878810,0.046064,0.069900,CAL FIRE / Kern County Fire / State Parks / US...,Kern County Fire
9,Los Padres National Forest,34.537650,-118.754950,0.127284,0.149062,Los Angeles County Fire,Unified Command LA County Fire and USFS


The above dataset now has the closets 2 neighboring admin units with the distances using K-D tree.

## Ball Tree

In [ ]:
# converting degree coordinates into radian values
for column in X[["Latitude", "Longitude"]]:
    rad = np.deg2rad(X[column].values)
    X[f'{column}_rad'] = rad
for column in y[["Latitude", "Longitude"]]:
    rad = np.deg2rad(y[column].values)
    y[f'{column}_rad'] = rad

In [ ]:
X.head()

,AdminUnit,Latitude,Longitude,Latitude_rad,Longitude_rad
0,Stanislaus National Forest/Yosemite National Park,37.857000,-120.086000,0.660729,-2.095896
1,USFS Angeles National Forest/Los Angeles Count...,34.585595,-118.423176,0.603633,-2.066874
2,CAL FIRE Riverside Unit / San Bernardino Natio...,33.709500,-116.728850,0.588342,-2.037303
3,Tahoe National Forest,39.120000,-120.650000,0.682773,-2.105740
4,Ventura County Fire/CAL FIRE,0.000000,0.000000,0.000000,0.000000


In [ ]:
y.head()

,AdminUnit,Latitude,Longitude,Latitude_rad,Longitude_rad
0,CAL FIRE Mendocino Unit,39.279833,-123.258319,0.685562,-2.151263
1,CAL FIRE Amador - El Dorado Unit,38.821280,-121.039180,0.677559,-2.112532
2,CAL FIRE San Benito-Monterey,36.648320,-121.032530,0.639634,-2.112416
3,CAL FIRE Santa Clara Unit/San Jose City Fire &...,37.176944,-121.686944,0.648860,-2.123838
4,CAL FIRE Shasta-Trinity Unit,40.741186,-122.649626,0.711068,-2.140640


In [ ]:
bt = BallTree(X[["Latitude_rad", "Longitude_rad"]].values, metric='haversine')

dist_bt, idx_bt = bt.query(y[["Latitude_rad", "Longitude_rad"]].values, k=2)

In [ ]:
distBT_df = pd.DataFrame(dist_bt, columns=['dist1', 'dist2'])
df_res_bt = y.merge(distBT_df, how='outer', left_index=True, right_index=True)
adminUnitBT_df = pd.DataFrame(idx_bt, columns=['unit1', 'unit2'])
df_res_bt = df_res_bt.merge(adminUnitBT_df, how='outer', left_index=True, right_index=True)
df_res_bt

,AdminUnit,Latitude,Longitude,Latitude_rad,Longitude_rad,dist1,dist2,unit1,unit2
0,CAL FIRE Mendocino Unit,39.279833,-123.258319,0.685562,-2.151263,0.000551,0.001356,700,631
1,CAL FIRE Amador - El Dorado Unit,38.821280,-121.039180,0.677559,-2.112532,0.000667,0.001170,557,836
2,CAL FIRE San Benito-Monterey,36.648320,-121.032530,0.639634,-2.112416,0.000202,0.002615,1147,539
3,CAL FIRE Santa Clara Unit/San Jose City Fire &...,37.176944,-121.686944,0.648860,-2.123838,0.000536,0.001035,813,592
4,CAL FIRE Shasta-Trinity Unit,40.741186,-122.649626,0.711068,-2.140640,0.000656,0.001555,985,1058
...,...,...,...,...,...,...,...,...,...
404,CAL FIRE / Riverside County Fire,33.827979,-117.499619,0.590410,-2.050755,0.000510,0.000726,753,1018
405,CAL FIRE Nevada-Yuba-Placer Unit,39.409722,-121.000556,0.687829,-2.111858,0.000785,0.001959,1027,115
406,Yolo County Fire Protection District,38.734634,-121.729691,0.676047,-2.124584,0.004098,0.004329,1119,1118
407,Camp Pendleton Marine Corps Base,33.351145,-117.403719,0.582087,-2.049081,0.000084,0.000895,697,692


In [ ]:
df_res_bt['adminUnit1'] = df_res_bt['unit1'].apply(get_adminunit)
df_res_bt['adminUnit2'] = df_res_bt['unit2'].apply(get_adminunit)
df_res_bt.drop(columns=['unit1','unit2'],inplace=True)
df_res_bt.head(15)

,AdminUnit,Latitude,Longitude,Latitude_rad,Longitude_rad,dist1,dist2,adminUnit1,adminUnit2
0,CAL FIRE Mendocino Unit,39.279833,-123.258319,0.685562,-2.151263,0.000551,0.001356,CAL FIRE Mendocino Unit,CAL FIRE Mendocinio Unit
1,CAL FIRE Amador - El Dorado Unit,38.821280,-121.039180,0.677559,-2.112532,0.000667,0.001170,CAL FIRE Amador-El Dorado Unit,CAL FIRE Amador-El Dorado Unit
2,CAL FIRE San Benito-Monterey,36.648320,-121.032530,0.639634,-2.112416,0.000202,0.002615,CAL FIRE San Benito-Monterey,CAL FIRE San Benito Monterey Unit
3,CAL FIRE Santa Clara Unit/San Jose City Fire &...,37.176944,-121.686944,0.648860,-2.123838,0.000536,0.001035,CAL FIRE Santa Clara Unit,CAL FIRE Santa Clara Unit
4,CAL FIRE Shasta-Trinity Unit,40.741186,-122.649626,0.711068,-2.140640,0.000656,0.001555,CAL FIRE Siskiyou Unit,"Unified Command: CAL FIRE Shasta-Trinity Unit,..."
5,CAL FIRE / Riverside County Fire,33.563490,-116.732032,0.585793,-2.037358,0.001355,0.001422,CAL FIRE/Riverside County Fire/San Bernardino ...,San Bernardino National Forest
6,Klamath National Forest,41.833590,-122.992720,0.730134,-2.146628,0.001657,0.002861,USFS Klamath National Forest,Rogue-River-Siskiyou National Forest
7,CAL FIRE Santa Clara Unit,37.327440,-121.109030,0.651487,-2.113751,0.001942,0.001957,CAL FIRE Santa Clara Unit,CAL FIRE Madera-Mariposa-Merced Unit
8,Kern County Fire Department,34.860220,-118.878810,0.608426,-2.074827,0.000694,0.001089,CAL FIRE / Kern County Fire / State Parks / US...,Kern County Fire
9,Los Padres National Forest,34.537650,-118.754950,0.602796,-2.072665,0.001838,0.002151,Los Angeles County Fire,Unified Command LA County Fire and USFS


Used the 'Haversine' distance as the metric (scikit-learn) to get better results.

# HNSW - **Hierarchical Navigable Small World Graphs**


MovieLens dataset from LightFM

In [ ]:
!pip install nmslib
import nmslib

In [ ]:
from lightfm.datasets import fetch_movielens
movielens = fetch_movielens()

In [ ]:
train = movielens['train']
test = movielens['test']

In [ ]:
model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)

model.fit_partial(train, item_features=movielens['item_features'], epochs=20 )

In [ ]:
from lightfm.evaluation import precision_at_k

In [ ]:
tr_preci = precision_at_k(model, train, k=10).mean()
test_preci = precision_at_k(model, test, k=10).mean()

print('Precision - for train data %.2f, for test data %.2f' %(tr_preci, test_preci))

Precision - for train data 0.73, for test data 0.10


In [ ]:
_, item_embeddings = model.get_item_representations(movielens['item_features']) # item embedding from item features

creating nmslib index using HSNW index with cosine similarity

In [ ]:
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [ ]:
def get_nearest_neighbors_hnsw(mv_id, index, n=8, print_output=True):
  nn = index.knnQuery(item_embeddings[mv_id], k=8)
  if print_output == True:
    print('Nearest to %s : \n' % movielens['item_labels'][mv_id])
  titles = [movielens['item_labels'][i] for i in nn[0]]
  if print_output == True:
    print('\n'.join(titles))

In [ ]:
get_nearest_neighbors_hnsw(50, nms_idx, n=8)

Nearest to Legends of the Fall (1994) : 

Legends of the Fall (1994)
Firm, The (1993)
Last of the Mohicans, The (1992)
Ghost (1990)
Walk in the Clouds, A (1995)
Fried Green Tomatoes (1991)
Boys on the Side (1995)
When a Man Loves a Woman (1994)


We can get the nearest neighbors that is similar movie names using the HNSW method with cosine similarity. I used k=8 to get 8 nearest neighbors for the movie "Legends of the Fall".

# References
1. https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6
2. https://making.lyst.com/lightfm/docs/datasets.html
3. https://faiss.ai/